In [1]:
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score


In [7]:
with open('../data/amazon/train_edges_true.txt') as f:
    train_edges_true = [(line.split(' ')[0], line.split(' ')[1]) for line in f.read().split('\n')]
train_edges_true = [(min(int(e[0]),int(e[1])),max(int(e[0]),int(e[1]))) for e in train_edges_true]

with open('../data/amazon/test_edges_true.txt') as f:
    test_edges_true = [(line.split(' ')[0], line.split(' ')[1]) for line in f.read().split('\n')]
test_edges_true = [(min(int(e[0]),int(e[1])),max(int(e[0]),int(e[1]))) for e in test_edges_true]

with open('../data/amazon/train_edges_false.txt') as f:
    train_edges_false = [(line.split(' ')[0], line.split(' ')[1]) for line in f.read().split('\n')]
train_edges_false = [(min(int(e[0]),int(e[1])),max(int(e[0]),int(e[1]))) for e in train_edges_false]

with open('../data/amazon/test_edges_false.txt') as f:
    test_edges_false = [(line.split(' ')[0], line.split(' ')[1]) for line in f.read().split('\n')]
test_edges_false = [(min(int(e[0]),int(e[1])),max(int(e[0]),int(e[1]))) for e in test_edges_false]

In [10]:
nodes_count = 334864

In [36]:
def filter(l):
    new_l = []
    for e in l:
        if e[0] < (334864/8) and e[1] < (334864/8):
            new_l.append(e)
    return new_l
train_edges_true = filter(train_edges_true)
train_edges_false = filter(train_edges_false)
test_edges_true = filter(test_edges_true)
test_edges_false = filter(test_edges_false)

In [47]:
len(train_edges_true)

10120

In [38]:
def write_edgelist(edgelist, path):
    with open(path, 'w+') as f:
        for e_idx, e in enumerate(edgelist):
            line =  str(e[0]) + ' ' + str(e[1])
            if e_idx != len(edgelist) - 1:
                line += '\n'
            f.write(line)
    print('file', path, 'created.')
write_edgelist(train_edges_true, '/Users/vahid/Desktop/train_true.txt')

file /Users/vahid/Desktop/train_true.txt created.


In [72]:
dic = {}
with open('/Users/vahid/Desktop/small_emb_3_3.emb') as f:
    for line in f.read().split('\n')[1:]:
        key = line.split(' ')[0]
        value = np.array([float(x) for x in line.split(' ')[1:]])
        dic[key] = value
print(len(dic))
for i in range(1 + int(nodes_count/8)):
    if str(i) not in dic:
        dic[str(i)] = np.ones([128])
print(len(dic))

14799
41860


In [73]:
train_X = []
train_y = []
test_X = []
test_y = []

false_z = 0
true_z = 0
for e in train_edges_true:
    train_X.append(np.multiply(dic[str(e[0])], dic[str(e[1])]))
    if np.multiply(dic[str(e[0])], dic[str(e[1])]).mean() > 1 - 1e-6:
        true_z += 1
    train_y.append(1)
for e in train_edges_false:
    train_X.append(np.multiply(dic[str(e[0])], dic[str(e[1])]))
    if np.multiply(dic[str(e[0])], dic[str(e[1])]).mean() > 1 - 1e-6:
        false_z += 1
    train_y.append(0)
    
for e in test_edges_true:
    test_X.append(np.multiply(dic[str(e[0])], dic[str(e[1])]))
    test_y.append(1)
for e in test_edges_false:
    test_X.append(np.multiply(dic[str(e[0])], dic[str(e[1])]))
    test_y.append(0)

train_X = np.array(train_X)
train_y = np.array(train_y)
test_X = np.array(test_X)
test_y = np.array(test_y)
train_X.shape,  train_y.shape, test_X.shape, test_y.shape

((20294, 128), (20294,), (14473, 128), (14473,))

In [74]:
true_z, false_z

(0, 4230)

In [75]:
%%time
edge_classifier = LogisticRegression(random_state=0, solver='lbfgs', max_iter=250)
edge_classifier.fit(train_X, train_y)

CPU times: user 1.08 s, sys: 29.4 ms, total: 1.11 s
Wall time: 358 ms


In [76]:
%%time
preds = edge_classifier.predict_proba(test_X)[:, 1]

CPU times: user 5.25 ms, sys: 738 µs, total: 5.99 ms
Wall time: 2.34 ms


In [77]:
test_roc = roc_auc_score(test_y, preds)
test_ap = average_precision_score(test_y, preds)
test_roc, test_ap

(0.9438028972795102, 0.9615585928377474)

In [78]:
test_y

array([1, 1, 1, ..., 0, 0, 0])

In [79]:
preds.mean()

0.6487882896273409

In [80]:
test_y.mean()

0.6992330546534927